In [132]:
import pandas as pd
import numpy as np
from km_plusplus import KMeans
from SVD import Matrix
import matplotlib.pyplot as plt
import math

In [133]:
def load_data(file_path):
    return pd.read_csv(file_path)

In [134]:
file_path = "./EncodedCombined2.csv"  # Replace with your dataset file path
data = load_data(file_path)

avg_rating = load_data("./Genre_rating.csv")
# Assuming the data matrix is constructed with users as rows and movies as columns
dataMat = data# Exclude first column (user IDs)
# data_matrix.drop(columns=["Zip-code","Title"],inplace=True)
dataMat

,UserID,MovieID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Gender,Age,Occupation
0,1,1193,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,10
1,1,661,3,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,1,1,10
2,1,914,3,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,1,10
3,1,3408,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,10
4,1,2355,5,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869365,4211,3791,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45,5
869366,4211,3806,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,45,5
869367,4211,3840,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45,5
869368,4211,3766,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,45,5


Add New User Details Here

In [135]:
newUserID=7000
newUserMovies={
    25 : 4.0,
    54 : 5.0,
    65: 3.0,
    1200 : 4.0,
    3200 : 5.0,
    2980 : 1.0,
    99 : 2.0,
    455 : 2.0 
}
newUserDetails = {
    'Age' : 18,
    'Gender' : 0,
    'Occupation' : 17
}


movieDetails=pd.read_csv("movies.csv")
movieDetails=movieDetails.set_index(keys='MovieID')
columns = ["UserID","MovieID","Rating","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical",
"Mystery","Romance","Sci-Fi","Thriller","War","Western" , "Gender", "Age", "Occupation"]

Rows=[]
for movieId,Rating in newUserMovies.items():
    newRow={}
    movie=movieDetails.loc[movieId]
    genres = movie['Genres'].split('|')
    newRow["UserID"]=newUserID
    newRow["MovieID"]=movieId
    newRow["Rating"]=Rating
    for genre in genres:
        newRow[genre]=int(1)
    for key, value in newUserDetails.items():
        newRow[key]=int(value)
    Rows.append(newRow)

newUserdf=pd.DataFrame(Rows,columns=columns)
newUserdf=newUserdf.fillna(int(0))

dataMat=pd.concat([dataMat,newUserdf])
dataMat=dataMat.drop_duplicates()
dataMat

,UserID,MovieID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Gender,Age,Occupation
0,1,1193,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
1,1,661,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
2,1,914,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1,10
3,1,3408,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
4,1,2355,5.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,7000,1200,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0,18,17
4,7000,3200,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,18,17
5,7000,2980,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,18,17
6,7000,99,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,18,17


In [136]:
dataMat["MovieID"]=dataMat["MovieID"].astype(np.int16)
dataMat["UserID"]=dataMat["UserID"].astype(np.int16)
for col in dataMat.columns:
    if(col!="MovieID" and col!="UserID"):
        dataMat[col]=dataMat[col].astype(np.int8)
print(dataMat.dtypes)

UserID         int16
MovieID        int16
Rating          int8
Action          int8
Adventure       int8
Animation       int8
Children's      int8
Comedy          int8
Crime           int8
Documentary     int8
Drama           int8
Fantasy         int8
Film-Noir       int8
Horror          int8
Musical         int8
Mystery         int8
Romance         int8
Sci-Fi          int8
Thriller        int8
War             int8
Western         int8
Gender          int8
Age             int8
Occupation      int8
dtype: object


In [137]:
df=dataMat
# Initialize an empty dictionary to store genre-wise ratings for each user
genre_ratings = {}

# Iterate over each row in the DataFrame
for _, row in df.iterrows():
    user_id = row['UserID']
    rating = row['Rating']
    genres = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
    
    # Iterate over each genre and update the genre_ratings dictionary
    for genre in genres:
        if genre not in genre_ratings:
            genre_ratings[genre] = {}
        if user_id not in genre_ratings[genre]:
            genre_ratings[genre][user_id] = {'total_rating': 0, 'count': 0}
        if row[genre] == 1:  # Check if the movie belongs to the genre
            genre_ratings[genre][user_id]['total_rating'] += rating
            genre_ratings[genre][user_id]['count'] += 1

# Initialize an empty dictionary to store the final DataFrame values
data = {}

# Iterate over each user and calculate the average rating for each genre
for user_id in set(df['UserID']):
    user_data = {}
    for genre, values in genre_ratings.items():
        if user_id in values and values[user_id]['count']>0:
            user_data[genre] = values[user_id]['total_rating'] / values[user_id]['count']
        else:
            user_data[genre] = np.nan
    data[user_id] = user_data

# Create the final DataFrame
result_df = pd.DataFrame.from_dict(data, orient='index')


Adding user details before SVD

In [138]:
# age=df['Age']
# gender=df['Gender']
# Occupation=df['Occupation']
# result_df['Age']=age
# result_df['Gender']=gender
# result_df['Occupation']=Occupation

In [142]:
data_matrix=result_df
data_matrix=data_matrix.sort_index()
data_matrix

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,4.200000,4.000000,4.111111,4.250000,4.142857,4.000000,NaN,4.428571,4.00,NaN,NaN,4.285714,NaN,3.666667,4.333333,3.666667,5.000000,NaN
2,3.500000,3.736842,NaN,NaN,3.560000,3.583333,NaN,3.898734,3.00,4.000000,3.000000,NaN,3.333333,3.708333,3.588235,3.483871,3.733333,4.333333
3,3.956522,4.000000,4.000000,4.000000,3.766667,NaN,NaN,4.000000,4.50,NaN,2.666667,4.000000,3.000000,3.800000,3.833333,3.800000,4.000000,4.666667
4,4.157895,3.833333,NaN,4.000000,NaN,5.000000,NaN,4.166667,4.50,NaN,4.333333,NaN,NaN,4.000000,3.555556,3.500000,3.333333,4.500000
5,2.612903,3.000000,4.000000,3.833333,3.410714,3.285714,3.666667,3.096154,NaN,4.000000,2.800000,3.333333,3.125000,3.100000,3.066667,2.846154,3.500000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3.000000,2.987952,3.911765,3.444444,3.203065,3.528302,3.909091,3.505376,3.00,4.058824,2.986486,3.709677,3.411765,3.352459,2.834320,3.142857,3.785714,3.642857
6037,3.642857,4.000000,4.000000,3.666667,3.576271,3.833333,4.000000,3.877551,4.25,3.444444,4.111111,4.000000,3.692308,3.681818,3.692308,3.705882,4.000000,3.750000
6038,3.000000,4.000000,3.666667,3.000000,3.833333,NaN,NaN,3.888889,NaN,NaN,2.500000,NaN,NaN,4.166667,4.000000,NaN,4.000000,NaN
6040,2.976190,2.818182,3.000000,4.000000,3.274510,3.920000,4.500000,3.821622,3.50,4.000000,2.590909,4.000000,4.454545,3.488889,3.473684,3.926829,3.695652,4.000000


In [154]:
user_ids = set(df['UserID'])
user_ids=sorted(user_ids)
user_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 35,
 36,
 37,
 38,
 39,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 169,
 170,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,

In [155]:
def get_non_zero_eigenvalues(Sigma):
    # Extract diagonal elements of Sigma
    diagonal_elements = np.diag(Sigma)

    # Filter out values not close to zero
    non_zero_eigenvalues = [eigenvalue for eigenvalue in diagonal_elements if not math.isclose(eigenvalue, 0)]

    return non_zero_eigenvalues


In [156]:
avgdict = avg_rating.set_index('Genre')['Avg. Rating'].to_dict()
avg_dict = {key: float(value)  for key, value in avgdict.items() if value != 'Avg. Rating'}

# A = data_matrix.pivot(index='UserID', columns='Genre', values='Rating')
A_mat = data_matrix
A_watched=dataMat.pivot(index='UserID', columns='MovieID', values='Rating')
# print(avg_dict)
A_mat= A_mat.fillna(avg_dict)
A_np=A_mat.values
A_np

array([[4.2       , 4.        , 4.11111111, ..., 3.66666667, 5.        ,
        3.63777015],
       [3.5       , 3.73684211, 3.68486822, ..., 3.48387097, 3.73333333,
        4.33333333],
       [3.95652174, 4.        , 4.        , ..., 3.8       , 4.        ,
        4.66666667],
       ...,
       [3.        , 4.        , 3.66666667, ..., 3.57046605, 4.        ,
        3.63777015],
       [2.97619048, 2.81818182, 3.        , ..., 3.92682927, 3.69565217,
        4.        ],
       [4.        , 2.        , 3.68486822, ..., 4.        , 4.        ,
        3.63777015]])

In [157]:
A_mat=A_mat.round(2)
# print(A_mat.dtypes)
A_mat

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,4.20,4.00,4.11,4.25,4.14,4.00,3.93,4.43,4.00,4.08,3.22,4.29,3.67,3.67,4.33,3.67,5.00,3.64
2,3.50,3.74,3.68,3.42,3.56,3.58,3.93,3.90,3.00,4.00,3.00,3.67,3.33,3.71,3.59,3.48,3.73,4.33
3,3.96,4.00,4.00,4.00,3.77,3.71,3.93,4.00,4.50,4.08,2.67,4.00,3.00,3.80,3.83,3.80,4.00,4.67
4,4.16,3.83,3.68,4.00,3.52,5.00,3.93,4.17,4.50,4.08,4.33,3.67,3.67,4.00,3.56,3.50,3.33,4.50
5,2.61,3.00,4.00,3.83,3.41,3.29,3.67,3.10,3.45,4.00,2.80,3.33,3.12,3.10,3.07,2.85,3.50,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3.00,2.99,3.91,3.44,3.20,3.53,3.91,3.51,3.00,4.06,2.99,3.71,3.41,3.35,2.83,3.14,3.79,3.64
6037,3.64,4.00,4.00,3.67,3.58,3.83,4.00,3.88,4.25,3.44,4.11,4.00,3.69,3.68,3.69,3.71,4.00,3.75
6038,3.00,4.00,3.67,3.00,3.83,3.71,3.93,3.89,3.45,4.08,2.50,3.67,3.67,4.17,4.00,3.57,4.00,3.64
6040,2.98,2.82,3.00,4.00,3.27,3.92,4.50,3.82,3.50,4.00,2.59,4.00,4.45,3.49,3.47,3.93,3.70,4.00


In [158]:
A_watched

MovieID,1,2,3,4,5,6,7,8,9,10,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
A = Matrix(A_mat)
U,Sigma,V = A.svd()

non_zero_diag = get_non_zero_eigenvalues(Sigma)
print("This is nonzero diag")
print(non_zero_diag)
U_red,Sigma_red,V_red = A.randomized_svd(len(non_zero_diag))  # Number of components for SVD 

WARN: no converge 

Outputs

[[ 0.0640952   0.02104759 -0.00176078 ...  0.          0.
   0.        ]
 [ 0.05753216 -0.00981722  0.00033392 ...  0.          0.
   0.        ]
 [ 0.06152262 -0.00476696  0.02331117 ...  0.          0.
   0.        ]
 ...
 [ 0.05815257  0.01156834  0.00377953 ...  0.          0.
   0.        ]
 [ 0.05795658 -0.02247364  0.00476372 ...  0.          0.
   0.        ]
 [ 0.05636217 -0.00053907  0.02827777 ...  0.          0.
   0.        ]]
[[267.16199935   0.           0.         ...   0.           0.
    0.        ]
 [  0.          32.07907854   0.         ...   0.           0.
    0.        ]
 [  0.           0.          32.26350029 ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]]
[[ 2.21707935e-01  2.11294720e-02 -2.94825796e-

In [160]:
np.savetxt('U_redFinal.txt', U_red)
np.savetxt('Sigma_redFinal.txt', Sigma_red)
np.savetxt('V_redFinal.txt', V_red)

In [161]:
U_red = np.loadtxt('U_redFinal.txt')
Sigma_red = np.loadtxt('Sigma_redFinal.txt')
V_red = np.loadtxt('V_redFinal.txt')
print(U_red.shape)
print(Sigma_red.shape)
print(V_red.shape)
# Reshape Sigma_red to a diagonal matrix
# Sigma_red = np.diag(Sigma_red)

(5330, 18)
(18, 18)
(18, 18)


In [162]:
def elbow_method(data, max_clusters=4 , skip_factor=1):
    distortions = []
    # Generate x-axis values with skip_factor increments
    x_values = range(skip_factor, max_clusters + 1, skip_factor)
    for i in x_values:
        km = KMeans(i)
        km.fit(data)
        distortions.append(km.get_cost())  # km.inertia_ gives the distortion (cost)
    # Plotting the elbow curve
    plt.plot(x_values, distortions, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Distortion')
    plt.title('Elbow Method')

# Assuming your data is in V_red_T
# Perform Elbow Method
new_mat = U_red
new_mat

array([[ 0.0640952 ,  0.02104759, -0.00176078, ..., -0.03274634,
        -0.00420459, -0.00288829],
       [ 0.05753216, -0.00981722,  0.00033392, ...,  0.01222886,
        -0.00419559,  0.00062669],
       [ 0.06152262, -0.00476696,  0.02331117, ..., -0.00114132,
        -0.00034585,  0.00778545],
       ...,
       [ 0.05815257,  0.01156834,  0.00377953, ..., -0.00380383,
        -0.00743566, -0.00918772],
       [ 0.05795658, -0.02247364,  0.00476372, ..., -0.02014829,
         0.01205556, -0.0187019 ],
       [ 0.05636217, -0.00053907,  0.02827777, ...,  0.01146724,
        -0.0175905 , -0.05566197]])

In [163]:
user_index = {}
i =0
for x in user_ids:
    user_index[int(x)] = i 
    i+=1

In [164]:
new_mat_df=pd.DataFrame(new_mat)
# print(dataMat)
totalData=dataMat.copy()
totalData=totalData.set_index(keys="UserID")
gendermap={}
agemap={}
occupationmap={}
for x in user_ids:
    gendermap[user_index[x]]=np.unique(totalData.loc[x]['Gender'])[0]
    agemap[user_index[x]]=np.unique(totalData.loc[x]['Age'])[0]
    occupationmap[user_index[x]]=np.unique(totalData.loc[x]['Occupation'])[0]

new_mat_df['Age']=agemap
new_mat_df['Gender']=gendermap
new_mat_df['Occupation']=occupationmap
new_mat=new_mat_df.values
new_mat

array([[ 6.40951984e-02,  2.10475940e-02, -1.76077895e-03, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+01],
       [ 5.75321638e-02, -9.81722030e-03,  3.33919202e-04, ...,
         5.60000000e+01,  0.00000000e+00,  1.60000000e+01],
       [ 6.15226167e-02, -4.76696080e-03,  2.33111658e-02, ...,
         2.50000000e+01,  0.00000000e+00,  1.50000000e+01],
       ...,
       [ 5.81525664e-02,  1.15683385e-02,  3.77953274e-03, ...,
         5.60000000e+01,  1.00000000e+00,  1.00000000e+00],
       [ 5.79565829e-02, -2.24736401e-02,  4.76372366e-03, ...,
         2.50000000e+01,  0.00000000e+00,  6.00000000e+00],
       [ 5.63621653e-02, -5.39074436e-04,  2.82777700e-02, ...,
         1.80000000e+01,  0.00000000e+00,  1.70000000e+01]])

In [165]:
new_mat_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,Age,Gender,Occupation
0,0.064095,0.021048,-0.001761,-0.007433,0.021112,0.001234,-0.000671,-0.004624,-0.006014,0.007826,...,0.008422,0.002268,-0.015101,-0.005037,-0.032746,-0.004205,-0.002888,1,1,10
1,0.057532,-0.009817,0.000334,0.006523,0.016379,0.009757,0.004935,-0.022806,-0.004738,-0.000363,...,0.006104,-0.000106,-0.017017,-0.004237,0.012229,-0.004196,0.000627,56,0,16
2,0.061523,-0.004767,0.023311,-0.014714,0.006420,0.019878,-0.000320,-0.009952,-0.010928,0.009456,...,0.010582,0.004698,-0.027634,-0.023181,-0.001141,-0.000346,0.007785,25,0,15
3,0.062913,-0.031043,-0.010132,-0.023508,-0.009129,-0.001982,-0.010489,-0.012491,-0.002452,0.010268,...,-0.008084,-0.016284,0.002358,-0.013182,0.014127,-0.012716,0.030868,45,0,7
4,0.053155,-0.015231,0.016860,0.014596,-0.011423,0.001500,-0.011258,0.002580,-0.015854,-0.003017,...,0.003393,0.002313,-0.016980,-0.017744,-0.008822,-0.010561,-0.002072,25,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5325,0.054342,-0.008540,0.002191,0.011283,0.002344,-0.004023,-0.010548,-0.009483,-0.011719,-0.011024,...,0.007833,0.006124,0.003928,-0.006849,-0.001757,-0.003339,-0.007659,25,1,15
5326,0.060716,-0.001177,-0.001802,-0.013087,-0.003162,-0.014645,0.014321,-0.003425,-0.003445,0.007433,...,-0.011226,0.006288,-0.002908,-0.011754,-0.004254,-0.002278,0.017237,45,1,1
5327,0.058153,0.011568,0.003780,0.002062,-0.000521,0.024607,0.016792,-0.022030,-0.001999,-0.011780,...,-0.001179,0.002664,-0.013012,0.012858,-0.003804,-0.007436,-0.009188,56,1,1
5328,0.057957,-0.022474,0.004764,0.001545,-0.000599,0.007762,-0.013934,0.009262,0.012048,-0.010505,...,-0.000804,-0.003429,-0.007178,0.012131,-0.020148,0.012056,-0.018702,25,0,6


In [ ]:
# elbow_method(new_mat,120,3)

In [166]:
km = KMeans(75)
km.fit(new_mat)

In [167]:
# Recommend movies for a particular user
def recommend_movies(user_id, data, users_watched,user_idx,cluster_labels):
    user_cluster = cluster_labels[user_idx[user_id]]
    # print(user_cluster)
    cluser_users = []
    for row,cluster in cluster_labels.items():
        if cluster == user_cluster and row!=user_idx[user_id]:
            cluser_users.append(users_watched.iloc[row]) 

    rating_movie = {}
    movie_rating_count={}
    for df in cluser_users:
        for key,value in df[df.notnull()].items():
            if int(key) not in rating_movie:
                rating_movie[int(key)] = 0
                movie_rating_count[int(key)] = 0
            
            rating_movie[int(key)]+=float(value)
            movie_rating_count[int(key)]+=1

    for key,value in movie_rating_count.items():
        if(value>0):
            rating_movie[key]=rating_movie[key]/float(value)

    final_list = sorted(rating_movie.items(), key=lambda x: x[1],reverse=True)

    movies=pd.read_csv("movies.csv")
    movies=movies.set_index(keys=['MovieID'])
    recommendations=[]
    for x in final_list:
        movie=movies.loc[x[0]]
        recommendations.append([x[0],x[1],movie['Title'],movie['Genres']])

    return recommendations

In [168]:
cluster_labels = km.get_pointCluster()

print(f"Cluster Labels: {cluster_labels}")
print(f"User_Index : {user_index}")
# User ID for which we want to recommend movies
user_id = newUserID  # Replace with the desired user ID

# Recommend movies for the user
recommended_movies = recommend_movies(user_id, new_mat,A_watched,user_index,cluster_labels)
print("Recommended movies for User", user_id)
for movie in recommended_movies[:5]:  # Display top 10 recommended movies
    print(f"Title : {movie[2]} , MovieID : {movie[0]} , Rating : {movie[1]} , Genres : {movie[3]}")  # Assuming the first column is movie titles


Cluster Labels: {0: 12, 1: 28, 2: 27, 3: 32, 4: 73, 5: 46, 6: 63, 7: 71, 8: 52, 9: 19, 10: 42, 11: 71, 12: 9, 13: 29, 14: 21, 15: 30, 16: 12, 17: 65, 18: 66, 19: 66, 20: 29, 21: 45, 22: 29, 23: 10, 24: 42, 25: 0, 26: 0, 27: 44, 28: 56, 29: 9, 30: 13, 31: 54, 32: 30, 33: 45, 34: 30, 35: 54, 36: 71, 37: 39, 38: 72, 39: 74, 40: 45, 41: 13, 42: 41, 43: 42, 44: 12, 45: 45, 46: 21, 47: 33, 48: 60, 49: 74, 50: 42, 51: 67, 52: 21, 53: 52, 54: 38, 55: 45, 56: 4, 57: 33, 58: 52, 59: 69, 60: 45, 61: 42, 62: 45, 63: 65, 64: 45, 65: 18, 66: 12, 67: 0, 68: 45, 69: 9, 70: 3, 71: 52, 72: 42, 73: 45, 74: 45, 75: 12, 76: 65, 77: 9, 78: 44, 79: 62, 80: 0, 81: 30, 82: 52, 83: 52, 84: 27, 85: 38, 86: 0, 87: 12, 88: 35, 89: 30, 90: 60, 91: 32, 92: 71, 93: 48, 94: 33, 95: 39, 96: 71, 97: 8, 98: 42, 99: 18, 100: 27, 101: 41, 102: 42, 103: 52, 104: 52, 105: 52, 106: 35, 107: 12, 108: 10, 109: 0, 110: 30, 111: 53, 112: 44, 113: 8, 114: 14, 115: 56, 116: 24, 117: 10, 118: 35, 119: 45, 120: 52, 121: 45, 122: 4, 1

In [169]:
df=A_watched.loc[user_id]
result_map = {}

for key,value in df[df.notnull()].items():
    result_map[key] = value

movies=pd.read_csv("movies.csv")
movies=movies.set_index(keys=['MovieID'])
result_map=dict(sorted(result_map.items(), key=lambda item: item[1],reverse=True))

movies_currently_watched=[]
for movieID,rating in result_map.items():
    movie=movies.loc[movieID]
    movies_currently_watched.append([movieID,rating,movie['Title'],movie['Genres']])

print("Movies Watched by User :", user_id)
for movie in movies_currently_watched:  # Display top 10 recommended movies
    print(f"Title : {movie[2]} , MovieID : {movie[0]} , Rating : {movie[1]} , Genres : {movie[3]}")  # Assuming the first column is movie titles

Movies Watched by User : 7000
Title : Big Green, The (1995) , MovieID : 54 , Rating : 5.0 , Genres : Children's|Comedy
Title : Last Detail, The (1973) , MovieID : 3200 , Rating : 5.0 , Genres : Comedy|Drama
Title : Leaving Las Vegas (1995) , MovieID : 25 , Rating : 4.0 , Genres : Drama|Romance
Title : Aliens (1986) , MovieID : 1200 , Rating : 4.0 , Genres : Action|Sci-Fi|Thriller|War
Title : Bio-Dome (1996) , MovieID : 65 , Rating : 3.0 , Genres : Comedy
Title : Heidi Fleiss: Hollywood Madam (1995) , MovieID : 99 , Rating : 2.0 , Genres : Documentary
Title : Free Willy (1993) , MovieID : 455 , Rating : 2.0 , Genres : Adventure|Children's|Drama
Title : Men Cry Bullets (1997) , MovieID : 2980 , Rating : 1.0 , Genres : Drama
